# Proyecto de Análisis de Sentimientos con Web Scraping (IMDb)

# Introducción
Análisis de Sentimientos en Reviews de Películas - IMDb

Este proyecto consiste en realizar Web Scraping para extraer reseñas de películas desde IMDb, analizarlas mediante una API de análisis de sentimientos, y almacenar los resultados en una base de datos. Se automatiza además la búsqueda de películas mediante Selenium.

Objetivos:
- Practicar Web Scraping con BeautifulSoup y Selenium.
- Aplicar análisis de sentimientos automático.
- Integrar con bases de datos locales (SQLite).


 # Fase 1: Inicialización del Proyecto y Base de Datos

In [151]:
!pip install selenium

In [1]:
import sqlite3
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
from bs4 import BeautifulSoup
import random
import pandas as pd


In [3]:
#Función de conversión de sentimientos

def sentimiento_a_valor(etiqueta):
    mapa = {
        'P+': 2,
        'P': 1,
        'NEU': 0,
        'N': -1,
        'N-': -2
    }
    return mapa.get(etiqueta.upper(), 0)


In [5]:
#Crear base de datos local
import sqlite3

def crear_base_datos(peliculas="reviews.db"):
    conn = sqlite3.connect(peliculas)
    c = conn.cursor()
    c.execute('''CREATE TABLE IF NOT EXISTS reviews (
                    pelicula TEXT,
                    review TEXT,
                    sentimiento TEXT,
                    valor INTEGER
                )''')
    conn.commit()
    conn.close()

crear_base_datos()


# Fase 2: Web Scraping con Selenium + BeautifulSoup

In [331]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time

def buscar_pelicula(titulo):
    driver = webdriver.Chrome()
    driver.get("https://www.imdb.com/")
    time.sleep(2)

    # Aceptar cookies si aparece
    try:
        accept_button = driver.find_element(By.CSS_SELECTOR, 'button[data-testid="accept-button"]')
        accept_button.click()
        time.sleep(1)
    except:
        pass

    # Buscar película
    search_box = driver.find_element(By.NAME, "q")
    search_box.send_keys(titulo)
    search_box.send_keys(Keys.RETURN)
    time.sleep(3)

    # Buscar título exacto entre los resultados
    titulos = driver.find_elements(By.CSS_SELECTOR, 'a.ipc-metadata-list-summary-item__t')
    encontrado = False
    for t in titulos:
        if t.text.strip().lower() == titulo.strip().lower():
            t.click()
            encontrado = True
            break

    if not encontrado:
        print(f"No se encontró la película exacta: {titulo}")
        driver.quit()
        return None, None

    time.sleep(2)

    url_reviews = driver.current_url
    html = driver.page_source
    driver.quit()
    return url_reviews, html


In [333]:
url, html = buscar_pelicula("Happy Gilmore 2")
print("🔗 URL de reviews:", url)

🔗 URL de reviews: https://www.imdb.com/es-es/title/tt31868189/?ref_=fn_all_ttl_1


In [335]:
url, html = buscar_pelicula("Happy Gilmore 2")
print("🔗 html de reviews:", html)

🔗 html de reviews: <html lang="es-ES" xmlns:og="http://opengraphprotocol.org/schema/" xmlns:fb="http://www.facebook.com/2008/fbml" class=" scriptsOn"><head><script async="" src="https://sb.scorecardresearch.com/beacon.js"></script><script async="" src="https://images-na.ssl-images-amazon.com/images/I/216YVwoRFDL.js" crossorigin="anonymous"></script><meta charset="utf-8"><meta name="viewport" content="width=device-width"><script>if(typeof uet === 'function'){ uet('bb', 'LoadTitle', {wb: 1}); }</script><title>Happy Gilmore 2 (2025) - IMDb</title><meta name="description" content="Happy Gilmore 2: Dirigido por Kyle Newacheck. Con Adam Sandler, Julie Bowen, Christopher McDonald, Benny Safdie. Revisa la carrera de golf de Happy Gilmore tras su victoria en el Tour Championship." data-id="main"><meta name="keywords" content="Reseñas, Horarios, DVD, Fotos, Calificaciones de los Usuarios, Sinopsis, Tráileres, Créditos"><meta name="google-site-verification" content="0cadf7898134e79b"><meta name="

In [337]:
#Extraer reviews
from bs4 import BeautifulSoup

def extraer_reviews(html):
    soup = BeautifulSoup(html, 'html.parser')
    elementos = soup.find_all('span', class_='ipc-rating-star--rating')
    ratings = [el.text.strip() for el in elementos]
    return [ratings[0].replace(",",".")]

In [339]:
extraer_reviews(html)

['6.3']

In [341]:
#Extraer reviews
from bs4 import BeautifulSoup

def extraer_reviews(html):
    soup = BeautifulSoup(html, 'html.parser')
    elementos = soup.find_all('span', class_='sc-4dc495c1-1 lbQcRY')
    ratings = [el.text.strip() for el in elementos]
    return [ratings[0].replace(",",".")]

In [343]:
extraer_reviews(html)

['6.3']

# Fase 3: Análisis de Sentimientos con API + Almacenamiento


In [345]:
# Simulación del análisis
import random

def simular_api_sentimiento(texto):
    etiquetas = ['P+', 'P', 'NEU', 'N', 'N-']
    return random.choice(etiquetas)


In [347]:

def insertar_reviews(pelicula, reviews, db="reviews.db"):
    conn = sqlite3.connect(db)
    c = conn.cursor()

    for valor in reviews:
        sentimiento = simular_api_sentimiento(reviews)
        valo_sent = sentimiento_a_valor(sentimiento)

        c.execute("DELETE FROM reviews WHERE review = ?", (valor,))
        
        c.execute("INSERT INTO reviews VALUES (?, ?, ?, ?)",
                  (pelicula, valor, sentimiento, valo_sent))

    conn.commit()
    conn.close()

In [349]:
# Fase Final: Ejecutar el pipeline completo

pelicula = "Happy Gilmore 2"
url, html = buscar_pelicula(pelicula)
reviews = extraer_reviews(html)
insertar_reviews(pelicula, reviews, db="reviews.db")
print(f"{len(reviews)} reviews de '{pelicula}' procesadas y almacenadas.")


1 reviews de 'Happy Gilmore 2' procesadas y almacenadas.


In [351]:
# Visualización 
import pandas as pd

def ver_resultados(db="reviews.db"):
    conn = sqlite3.connect(db)
    df = pd.read_sql_query("SELECT * FROM reviews", conn)
    conn.close()
    return df

df_reviews = ver_resultados()
df_reviews


,pelicula,review,sentimiento,valor
0,Inception,Rating automático: 8.2,N/A,8.2
1,Inception,-1,N,8.2
2,Happy Gilmore 2,6.3,P,1.0
